In [1]:
!pip install gensim
!pip install StellarGraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 435 kB 22.4 MB/s 
     |████████████████████████████████| 462 kB 48.7 MB/s 


In [2]:
import numpy as np
import pandas as pd
import networkx as nx
from stellargraph.data import BiasedRandomWalk
from stellargraph import StellarGraph
from gensim.models import Word2Vec
import csv
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
from torch.autograd import Variable

Extract information from train.csv

In [3]:
#Get train.csv data
df = pd.read_csv('train.csv', header=None, sep='\n')
df = df[0].str.split(',', expand=True) 

In [4]:
#Get ingredient id
df2 = pd.read_fwf('node_ingredient.csv', header=None)
idToNode = {}
for index , row in df2.iterrows():
  idToNode[index] = row[0]

In [64]:
idToNode

{0: 'coca-cola',
 1: 'vegan butter',
 2: 'sourdough rolls',
 3: 'reduced sodium refried beans',
 4: 'ramen noodles',
 5: 'crumbled corn bread',
 6: 'japanese breadcrumbs',
 7: 'toasted shredded coconut',
 8: 'chinese spinach',
 9: "Hellmann's® Real Mayonnaise",
 10: 'ducklings',
 11: 'basil olive oil',
 12: 'white baking bar',
 13: 'rye whiskey',
 14: 'mushroom broth',
 15: 'meat loaf mix',
 16: 'cocktail sauce',
 17: 'asparagus spears',
 18: 'nonfat greek yogurt',
 19: 'cabernet sauvignon',
 20: 'low sodium tomato sauce',
 21: 'shredded colby',
 22: 'dried dillweed',
 23: 'frozen potatoes',
 24: 'hot dog bun',
 25: 'balm leaves',
 26: 'lo mein noodles',
 27: 'green split peas',
 28: 'celery leaves',
 29: 'butterflied leg of lamb',
 30: 'refrigerated piecrusts',
 31: 'cajun spice mix',
 32: 'finely chopped onion',
 33: 'chili bean sauce',
 34: 'brown rice',
 35: 'cutlet',
 36: 'beef jerky',
 37: 'lemon grass',
 38: 'ground walnuts',
 39: 'kinchay',
 40: 'macarons',
 41: 'crème de cassi

In [5]:
#Create dict of dict where the first key is ingredient id the value is another dict 
#with key are nodes he shares receipt with and value the number of receipt they have in common
ingredientAdjancy = {}
ingredientNbR = {}
for index , row in df.iterrows():
    for i in row.values:
        if i is not None and i.isdigit() and int(i) not in ingredientAdjancy.keys():
            ingredientAdjancy[int(i)] = {}
        for y in row.values:
            if i is not None and  y is not None and i.isdigit() and y.isdigit():
                if i != y:
                    if int(y) not in ingredientAdjancy[int(i)].keys():
                        ingredientAdjancy[int(i)][int(y)] = 1
                    else:
                        ingredientAdjancy[int(i)][int(y)] += 1
        if i is not None and i.isdigit():
            if int(i) not in ingredientNbR.keys():
                ingredientNbR[int(i)] = 1
            else:
                ingredientNbR[int(i)] += 1

In [6]:
import csv
fh = open('edgeListWeight.csv','w')
writer = csv.writer(fh)
for i in ingredientAdjancy.keys():
    for y in ingredientAdjancy[i].keys():
        t = str(i)+'\t'+str(y)+'\t'+str(ingredientAdjancy[i][y])
        writer.writerow([t])
fh.close()

In [7]:
import csv
fh = open('edgeList.csv','w')
writer = csv.writer(fh)
for i in ingredientAdjancy.keys():
    for y in ingredientAdjancy[i].keys():
        t = str(i)+'\t'+str(y)
        writer.writerow([t])
fh.close()

Create the graph

In [8]:
Gw = nx.read_edgelist('edgeListWeight.csv', nodetype=int, data=(('weight',float),))
G = nx.read_edgelist('edgeList.csv', nodetype=int, data=(('weight',float),))

1st Embedding:

Use Node2vec -> first need to generate weighted random walk. Then use Word2Vec to generate the embedding. Generate random walk take quite some time, I already generate 4 of them using different paramters that should adjust the balance between sutructural and local importance. This might be improved by trying different set of hyperparameters.

In [10]:
#Use StellarGraph library to generate the weighted random walk
GwS = StellarGraph.from_networkx(Gw)
rw = BiasedRandomWalk(GwS)
weighted_walks = rw.run(
        nodes=Gw.nodes(), # root nodes
        length=100,    # maximum length of a random walk
        n=10,          # number of random walks per root node
        p=1,         # Defines (unormalised) probability, 1/p, of returning to source node
        q=10,         # Defines (unormalised) probability, 1/q, for moving away from source node
        weighted=True, #for weighted random walks
        seed=42        # random seed fixed for reproducibility
    )

In [ ]:
#Save the random walk
import csv

fh = open('WRandomWalkp1q10.csv','w')
writer = csv.writer(fh)
for i in range(5860):
  for y in range(100):
    t = str(GwS.nodes()[i])+" "+str(weighted_walks[i*10+y])
    writer.writerow([t])
fh.close()

In [ ]:
#Import random walk
import ast
rp1q10 = pd.read_fwf('WRandomWalkp1q10.csv', delimiter= " ")

rp1q10_list = []
for index, value in rp1q10.iterrows():
  t = False
  s = ""
  for i in value[0]:
    if i == "[":
      t = True
    if i == '"' and t:
      t = False
    if t:
      s+=i
  rp1q10_list.append(ast.literal_eval(s)) 
  

In [ ]:
#preprocess Word2Vec
rp1q10W = []
for i in rp1q10_list:
  rp1q10W.append([str(x) for x in i])

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

model = Word2Vec(sentences=rp1q10W, vector_size=100, window=5, min_count=1, workers=4)

We create 2 embedding with node2vec -> q = 1, p = 2; q = 1, p =10

2nd embedding

Adjacency-based Similarity approximate using SVD, with and without weight

In [ ]:
from sklearn.decomposition import  TruncatedSVD
#no weight
A = nx.adjacency_matrix(G) 

svd = TruncatedSVD(n_components=128)
X = svd.fit_transform(A);

In [ ]:
emb = {}
y = 0
for i in Gw.nodes():
    emb[i] = X[y] 
    y += 1


In [ ]:
df_s = pd.DataFrame(data=list(emb.items()),columns=["node", "embedding"])
df_s.to_csv("SVD128.csv")

In [ ]:
#weight
A = nx.adjacency_matrix(Gw) 

svd = TruncatedSVD(n_components=128)
X = svd.fit_transform(A);

In [ ]:
emb = {}
y = 0
for i in Gw.nodes():
    emb[i] = X[y] 
    y += 1


In [ ]:
df_s = pd.DataFrame(data=list(emb.items()),columns=["node", "embedding"])
df_s.to_csv("SVDW128.csv")

6 embedding have been created using this method, from 32, 64, 128 dimensions for weighted and non weighted graph

3rd emebedding

Emebdding layer of a bidirectionnal LSTM train on the classification task

utility function

In [ ]:
def make_binary_data(cookbook, ingredient_list):
    # X is a binary matrix with recipe rows and ingredients columns, 
    # where (recipe, ingredients) is 1 if the ingredient is present in the recipe

    n_recipies = len(cookbook)
    n_ingredients = len(ingredient_list)

    X = np.zeros((n_recipies, n_ingredients))
    y = np.zeros(n_recipies)
    for idx, recipe in enumerate(cookbook):
        y[idx] = recipe['kitchen_id']
        for ingredient_id in recipe['ingredients']:
            X[idx][ingredient_id] = 1
            
    return X,y


def make_recipe_embedding_data(cookbook, embedding, avg=False):
    # Make the data. X is a matrix with recipe rows and embedding dimension columns, 
    # where every row is the average of the embeddings in the recipes

    n_recipies = len(cookbook)
    embedding_dim = len(embedding[0])

    X = np.zeros((n_recipies, embedding_dim))
    y = np.zeros(n_recipies)
    for idx, recipe in enumerate(cookbook):
        embedding_avg = np.zeros(embedding_dim) 
        ingredient_count = len(recipe['ingredients'])
        y[idx] = recipe['kitchen_id']
        for ingredient_id in recipe['ingredients']:
            if avg:
                embedding_avg += embedding[ingredient_id] / ingredient_count
            else:
                embedding_avg += embedding[ingredient_id]
            
        X[idx, :] = embedding_avg
            
    return X,y


def make_ingredient_embedding_data(embedding):
    # A matrix with as rows the ingredients and as columns the embedding
    matrix = np.empty( (len(embedding), len(embedding[0])) )
    for ingredient_idx, embedding_vector in embedding.items():
        matrix[ingredient_idx, :] = embedding_vector
    return matrix

In [ ]:
from csv import reader
import pandas as pd
import torch

__kitchen_list = []


def get_cookbook_train():

    cookbook_train = []

    with open('train.csv', 'r') as file:

        csv_reader = reader(file, delimiter=",")

        for i, row in enumerate(csv_reader):
            
            kitchen = row[-1]
            ingredient_strings = row[:-1]
            ingredients = [ int(s) for s in ingredient_strings ]
            
            cookbook_train.append({
                'recipe_id': i,
                'ingredients': ingredients,
                'kitchen_name': kitchen
            })      

    fill_kitchen_list(cookbook_train)
    add_kitchen_id_to_cookbook(cookbook_train)
    return cookbook_train


def get_cookbook_valid_question():

    cookbook_valid_question = []

    with open('validation_classification_question.csv', 'r') as file:

        csv_reader = reader(file, delimiter=",")

        for i, row in enumerate(csv_reader):
            ingredients = [ int(s) for s in row ]
            cookbook_valid_question.append({
                'recipe_id': i,
                'ingredients': ingredients,
                'kitchen_name': "UNKNOWNKITCHEN",
                'kitchen_id': "-999"
            })     

    return cookbook_valid_question


       
def get_cookbook_valid_answer():

    cookbook_valid_answer = []

    with open('validation_classification_answer.csv', 'r') as file:

        csv_reader = reader(file, delimiter=",")

        for i, row in enumerate(csv_reader):        
            kitchen = row[0]        
            cookbook_valid_answer.append({
                'recipe_id': i,
                'ingredients': [],
                'kitchen_name': kitchen
            })

    fill_kitchen_list(cookbook_valid_answer)
    add_kitchen_id_to_cookbook(cookbook_valid_answer)
    return cookbook_valid_answer


def get_ingredient_list():

    node_ingredient = pd.read_fwf('node_ingredient.csv', header=None)
    ingredient_list = [None] * node_ingredient.shape[0]
    for index , row in node_ingredient.iterrows():
        ingredient_list[index] = row[0]

    return ingredient_list


def fill_kitchen_list(cookbook):

    for recipe in cookbook:
        if recipe['kitchen_name'] not in __kitchen_list:
            __kitchen_list.append(recipe['kitchen_name'])



def add_kitchen_id_to_cookbook(cookbook):   

    for recipe in cookbook:
        kitchen_id = __kitchen_list.index(recipe['kitchen_name'])
        recipe['kitchen_id'] = kitchen_id

In [ ]:
cookbook_train = get_cookbook_train()
cookbook_valid_question = get_cookbook_valid_question()
cookbook_valid_answer = get_cookbook_valid_answer()
ingredient_list = get_ingredient_list()


Preprocessing

In [ ]:
m = 0
minI = 60
maxI = 0
dicNb = {}
for i in cookbook_train:
  if len(i["ingredients"]) > m:
    m = len(i["ingredients"])
  for y in i["ingredients"]:
    if y not in dicNb.keys():
      dicNb[y] = 1
    else:
      dicNb[y] += 1
    if y < minI:
      minI = y
      l = i["ingredients"]
      print(l)
    if y > maxI:
      maxI = y

for i in range(6714):
  if i not in dicNb.keys():
    dicNb[i] = 0
print(m,maxI,minI,l)

In [ ]:
m = 30
XtrainRnn = []
for i in cookbook_train:
  x = i["ingredients"].copy()
  x = sorted(x, key=lambda y: dicNb[y],reverse=True)
  if len(x) > m:
    x = x[:m]
  while len(x) < m:
    x.append(6714)
  XtrainRnn.append(x)
  
XvalRnn = []
for i in cookbook_valid_question:
  x = i["ingredients"].copy()
  x = sorted(x, key=lambda y: dicNb[y],reverse=True)
  if len(x) > m:
    x = x[:m]
  while len(x) < m:
    x.append(6714)
  XvalRnn.append(x)

In [ ]:
YtrainRnn = []
dic = {}
u = 0
for i in cookbook_train:
  if i["kitchen_name"] not in dic.keys():
    dic[i["kitchen_name"]] = u
    u += 1
print(u)
for i in cookbook_train:
  YtrainRnn.append(dic[i["kitchen_name"]])

YvalRnn = []
for i in cookbook_valid_answer:
  YvalRnn.append(dic[i["kitchen_name"]])
YvalRnn

In [ ]:
tX = torch.tensor(XtrainRnn)
tY = torch.tensor(YtrainRnn)
vX = torch.tensor(XvalRnn)
vY = torch.tensor(YvalRnn)
train = TensorDataset(tX, tY)
#val = TensorDataset(val_features, val_targets)
train_loader = DataLoader(train, batch_size=32, shuffle=True)

In [ ]:
import torch.nn as nn
from torch.autograd import Variable

class LSTM(nn.Module):

  def __init__(self,voc_size, embedding_length,hidden_size,output_size):
    super(LSTM, self).__init__()
    self.emb = nn.Embedding(voc_size, embedding_length)
    self.lstm = nn.LSTM(embedding_length, hidden_size, 2,batch_first=True, bidirectional = True,dropout = 0.8)
    self.label = nn.Linear(hidden_size, output_size)
    self.hidden_size = hidden_size
    self.drop = nn.Dropout(0.8)

  def forward(self,input):
    input = self.emb(input)
    _,(final_hidden_state, final_cell_state) = self.lstm(input)
    return self.label(self.drop(final_hidden_state[-1])) 


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = LSTM(6715,64,128,20)
cel = nn.CrossEntropyLoss()
n_epochs = 70
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model.to(device)

In [ ]:
for epoch in range(1, n_epochs + 1):
  batch_losses = []
  lmean = 0
  nb = 0
  for x_batch, y_batch in train_loader:
    pred = model(x_batch.to(device))
    loss = cel(pred,y_batch.to(device))
    optimizer.zero_grad() 
    loss.backward() 
    optimizer.step()

    lmean += loss.item()
    nb += 1
  if epoch%20 == 1:
    model.eval()
    acc = 0
    for i in range(500):
      out = model(vX[i].to(device))
      c = out.max(0).indices.item()
      if c == vY[i].item(): #compare class
        acc += 1
      #print(c,vY[i].item())
        
    print(acc/500)
  if epoch%20 == 1:
    acc = 0
    for i in range(500):
      out = model(tX[i].to(device))
      c = out.max(0).indices.item()
      if c == tY[i].item(): #compare class
        acc += 1
      #print(c,vY[i].item())  
    print(acc/500)
    
  model.train()
  print(epoch,lmean/nb)

In [ ]:
model.eval()
acc = 0
for i in range(5000):
  out = model(vX[i].to(device))
  c = out.max(0).indices.item()
  if c == vY[i].item(): #compare class
    acc += 1
    #print(c,vY[i].item())
        
print(acc/5000)

In [ ]:
print(model.emb)

In [ ]:
embedding = model.emb.weight
embeddingL = embedding.tolist()
emb = {}
for i in range(6714):
  emb[i] = embeddingL[i]
  

In [ ]:
df_s = pd.DataFrame(data=list(emb.items()),columns=["node", "embedding"])
df_s.to_csv("LSTM64.csv")

2 embedding have been created using this method with 64 and 128 dimensions

4th embedding

Word2Vec were each receipts are "sentences"

In [ ]:
W2v = []
m = 30
for i in cookbook_train:
  x = []
  for y in i['ingredients']:
    x.append(str(y))
  if len(x) > m:
    x = x[:m]
  while len(x) < m:
    x.append('6714')
  W2v.append(x)
for i in cookbook_valid_question:
  x = []
  for y in i['ingredients']:
    x.append(str(y))
  if len(x) > m:
    x = x[:m]
  while len(x) < m:
    x.append('6714')
  W2v.append(x)

In [ ]:
embW = Word2Vec(sentences=W2v, 
                 min_count=1,
                 size=128,  
                 workers=4)

In [ ]:
import random

weightW = []
for i in range(6715):
  try:
    weightW.append(embW[str(i)])
  except:
    weightW.append(np.random.randn(128))
    continue  

Test of the cosin similarity using the different embedding / qualitative comparaison

In [52]:
import re

def import_embedding(filename):

    df = pd.read_csv(f'{filename}', delimiter=",")

    # Create an embedding dictionary from the file
    embedding = {}
    for index, row in df.iterrows():
        ingredient_id = row[1]
        embedding_string = row[2]
        embedding_formatted = re.findall(r'[\d|\.|e|\+|\-]+', embedding_string)
        embedding_array = np.asarray(embedding_formatted, dtype=float)
        embedding[ingredient_id] = embedding_array

        
    return embedding


In [99]:
from numpy import dot
from numpy.linalg import norm
import json

def compare(ingredient,emb,k):
  dic = {}
  print(idToNode[ingredient])
  a = emb[ingredient]
  for i in emb.keys():
    if i != ingredient:
        b = emb[i]
        cos_sim = dot(a, b)/(norm(a)*norm(b))
        dic[i] = cos_sim
  s = sorted(dic.items(), key=lambda x: x[1], reverse=True)
  s2 = sorted(dic.items(), key=lambda x: x[1])
  p = 0
  out = {}
  out2 = {}
  for i in s:
    out[idToNode[i[0]]] = i[1]
    p += 1
    if p == k:
      break
  p = 0
  for i in s2:
    out2[idToNode[i[0]]] = i[1]
    p += 1
    if p == k:
      break
  print("most similar = ",out)
  print("most different = ",out2)
  



In [79]:
svd128 = import_embedding('SVD128.csv')
svdw128 = import_embedding('SVDW128.csv')
lstm128 = import_embedding('LSTM128.csv')
node2vecp1q10 = import_embedding('Embp1q10.csv')
word2vec = import_embedding('Word2vec128.csv')

SVD128

In [100]:
compare(956,svd128,10)

oysters
most similar =  {'crabmeat': 0.8469109365400646, 'chicken livers': 0.8244797641938719, 'cooked ham': 0.813954434213496, 'crawfish': 0.8103423157178145, 'shrimp stock': 0.8037105166743104, 'shucked oysters': 0.8015814725655228, 'lump crab meat': 0.8000990658140146, 'onion tops': 0.795682673260433, 'file powder': 0.7864630290703688, 'old bay seasoning': 0.7859449566501695}
most different =  {'Ragu Classic Alfredo Sauce': -0.052614296013283025, 'Ragu Golden Veggie Fettuccine Pa': -0.052614296013283025, 'whole wheat crackers': -0.040382442831768076, 'wish bone red wine vinaigrett dr': -0.032272024000584834, 'huckleberries': -0.02586730426329719, 'crème de framboise': -0.02506376953526593, 'garlic olive oil': -0.023244014330829784, 'low-fat sweetened condensed milk': -0.022602938509779485, 'reduced fat ranch dressing': -0.014372735743185618, 'mini pepperoni slices': -0.014191607115810792}


In [101]:
compare(2945,svd128,10)

water
most similar =  {'salt': 0.8142271728940506, 'onions': 0.7850241341730618, 'sugar': 0.7757386323823957, 'garlic cloves': 0.7671398360055689, 'garlic': 0.7579657053453669, 'carrots': 0.742675974222317, 'vegetable oil': 0.7375092133974434, 'green onions': 0.7327644471599531, 'pepper': 0.7248302739119928, 'oil': 0.7195956525170696}
most different =  {'sago pearls': -0.020250042907392282, 'dried bonito': -0.0010406250251279849, 'Ragu Sauce': -1.8569913274766428e-05, 'Lipton Lemon Iced Tea Mix': 0.0009245875252110072, 'high-gluten flour': 0.0026286881923145472, 'lapsang': 0.00892165116502014, 'sour mix': 0.014118814865206355, 'Lipton® Iced Tea Brew Family Siz': 0.02100810736566502, 'red grapefruit': 0.023485172054981778, 'fine egg noodles': 0.024929448235820106}


In [102]:
compare(3934,svd128,10)

beef
most similar =  {'meat': 0.829434043348535, 'beef broth': 0.8163608810253355, 'pork': 0.8163372310965503, 'bell pepper': 0.8153092916853254, 'cabbage': 0.8081114260506583, 'red pepper': 0.8045089020248994, 'rice': 0.8004294514568273, 'sauce': 0.7993237576959582, 'boneless chicken breast': 0.7909842524959418, 'cooking oil': 0.7908474002843684}
most different =  {'blue curaçao': -0.061538232668525615, 'almond syrup': -0.053398767219905725, 'energy drink': -0.050828213605246385, 'cola-flavored carbonated beverag': -0.050828213605246385, 'sports drink': -0.04062882584941009, 'dry gin': -0.04046485142544161, 'lambrusco': -0.03910551100941281, 'crème de framboise': -0.03461722790945396, 'huckleberries': -0.032288788462974706, 'absinthe': -0.028074407270798678}


In [103]:
compare(71,svd128,10)

black tea
most similar =  {'tea bags': 0.7304371700032178, 'sugar syrup': 0.6893987638484804, 'orange peel': 0.6852856034325084, 'brown rice vinegar': 0.6763849894622718, 'ice cubes': 0.6737396484974221, 'grapefruit': 0.6730457013794638, 'black tea leaves': 0.6599950927039077, 'club soda': 0.650890050515717, 'ginger ale': 0.6464989810563478, 'navel oranges': 0.6455262140080678}
most different =  {'pot pie': -0.08751557737733431, 'quince paste': -0.07921214168175923, 'taleggio': -0.06403600100333896, 'capocollo': -0.0523326634915877, 'spicy salami': -0.0523326634915877, 'mie': -0.04834776913276861, 'crouton italian season': -0.0408342223917674, 'italian pizza crust': -0.037191864536221954, 'redfish fillet': -0.03320233110281212, 'Barilla Oven-Ready Lasagne': -0.033178984951356925}


SVD128 Weight

In [104]:
compare(956,svdw128,10)

oysters
most similar =  {'lump crab meat': 0.8850502389655671, 'ham': 0.8523209971290819, 'crabmeat': 0.851582816320692, 'old bay seasoning': 0.850613540090065, 'medium shrimp': 0.8484134444854421, 'chicken livers': 0.8483879198001534, 'shucked oysters': 0.847814463615135, 'creole seasoning': 0.8473782593331557, 'Tabasco Pepper Sauce': 0.8466083388397829, 'shrimp': 0.8424534943390364}
most different =  {'white creme de cacao': -0.04433701127691261, 'vanilla vodka': -0.04433701127691261, 'sherbet': -0.03315998921172729, 'nondairy whipped topping': -0.02670822701547808, 'salad leaves': -0.024105574204991858, 'Jell-O Gelatin Dessert': -0.022697448997070435, 'food paste color': -0.021558428029397027, 'Kraft Sun Dried Tomato Vinaigret': -0.02082588427040487, 'durum wheat flour': -0.018770599233270648, 'dried chipotle pepper': -0.01847854409797877}


In [105]:
compare(2945,svdw128,10)

water
most similar =  {'boiling water': 0.9420796214268267, 'hot water': 0.9415871032006851, 'chicken stock': 0.9341518810633602, 'chicken': 0.9261520575852513, 'meat': 0.9251654626833927, 'fat': 0.9216506775089136, 'lemon juice': 0.9195934563813172, 'chicken broth': 0.9186857286443013, 'black pepper': 0.9155421964041384, 'vegetables': 0.9151630672327901}
most different =  {'sago pearls': -0.00017666681375585815, 'diet dr. pepper': 0.0008790646340323525, 'dried bonito': 0.0013460450459441347, 'bénédictine': 0.0016123574074330554, 'Lipton Lemon Iced Tea Mix': 0.005972864591311979, 'huckleberries': 0.006797653519292029, 'Ragu Sauce': 0.006809008023348838, 'sour mix': 0.009364681747969276, 'wish bone red wine vinaigrett dr': 0.010580691665382407, 'spicy salami': 0.014329880213680642}


In [106]:
compare(3934,svdw128,10)

beef
most similar =  {'pork': 0.9475739522564488, 'cabbage': 0.9346679089098926, 'chili pepper': 0.9242377435328322, 'noodles': 0.9226533566713998, 'vinegar': 0.9194937408029673, 'stock': 0.9184612221869712, 'rice': 0.9168415474426531, 'meat': 0.9165430415131179, 'vegetables': 0.9151331811319108, 'mushrooms': 0.9117752783827203}
most different =  {'ground coffee': -0.01948836655200602, 'sliced fresh fruit': -0.013502374956880078, 'wish bone red wine vinaigrett dr': -0.008902698792702514, 'nondairy whipped topping': -0.008790244840903575, 'aged gouda': -0.008596850451528614, 'unsalted roasted pistachios': -0.007993135626930416, 'coffee low-fat frozen yogurt': -0.007885750650858097, 'Domino Confectioners Sugar': -0.007384095226308086, 'granular sucrolose sweetener': -0.007021310342091981, 'quince paste': -0.006836010672590416}


In [107]:
compare(71,svdw128,10)

black tea
most similar =  {'plums': 0.7000954918236041, 'ice cubes': 0.6970612179235776, 'tea bags': 0.6943385491732418, 'silver': 0.6906326381743418, 'orange': 0.6899130109348666, 'pork baby back ribs': 0.6782757516998599, 'green tea': 0.6765423896096482, 'grapefruit': 0.6698520354387545, 'ice': 0.6639159285154526, 'mint sprigs': 0.6588623999534171}
most different =  {'spicy salami': -0.10757703073668627, 'capocollo': -0.1067433617115797, 'taco seasoned cheese': -0.07294721605513857, 'shrimp chips': -0.06431526382237923, 'kumquats in syrup': -0.059275595195389365, 'red pepper hot sauce': -0.0501710184150049, 'Old El Paso™ mild red enchilada': -0.04983175171434783, 'chicken nugget': -0.04220838702892408, 'medium cheddar cheese': -0.03220291029014205, 'paratha': -0.028973985634553067}


LSTM128

In [108]:
compare(956,lstm128,10)

oysters
most similar =  {'non-dairy margarine': 0.3244311496649866, 'fresh veget': 0.2942491670837736, 'Holland House White Wine Vinegar': 0.28808550016097567, 'serrano peppers': 0.28705294183730745, 'white rice vinegar': 0.28612797371899307, 'pita bread': 0.274361295701365, 'catsup': 0.257620267701732, 'gelatin': 0.25707354805104193, 'whole wheat tortillas': 0.2526015107749431, 'squash seeds': 0.2518476192456978}
most different =  {'light cream cheese': -0.3374689844568287, 'beef sirloin': -0.3163949374291713, 'flan': -0.30986742691898034, 'gala apples': -0.2991590654480132, 'sunchokes': -0.259379104930181, 'pinenuts': -0.2547980771435051, 'pita chips': -0.25142631763184636, 'romana': -0.25127775867173513, 'hillshire farms low fat sausage': -0.24705649380583491, 'spring onions': -0.24638623637303908}


In [109]:
compare(2945,lstm128,10)

water
most similar =  {'marrow': 0.3436740640504066, 'jerk paste': 0.3003874582948378, 'Spice Islands Ground Cumin Seed': 0.2997214341920753, 'flanken short ribs': 0.28771234183964794, 'cook egg hard': 0.2784977698871533, 'Chinese sesame paste': 0.27551998632976404, 'green bellpepper': 0.2674178460073074, 'fresh orange': 0.26271258828120425, 'Tipo 00 flour': 0.25921699049820257, 'canadian bacon': 0.2562444581016512}
most different =  {'cremini mushrooms': -0.33276208547589237, 'Knorr® Beef Bouillon': -0.328877170127893, 'pancake batter': -0.32634327332295743, 'light sour cream': -0.30406312946884057, 'straw mushrooms': -0.2921767283659083, 'unsweetened applesauce': -0.28184560508977347, 'chocolate chip cookie mix': -0.27952601192314885, 'preserv raspberri seedless': -0.2790002651564464, 'sweet and sour mix': -0.27741514488773883, 'seeds': -0.270727263054195}


In [110]:
compare(3934,lstm128,10)

beef
most similar =  {'slider buns': 0.29444559983706325, 'won ton skins': 0.27685753730220614, 'beef bouillon powder': 0.2719513614975605, 'buckwheat groats': 0.26932244286595874, 'blackening seasoning': 0.26438335622146986, 'comino': 0.2603752903190379, 'Mazola Canola Oil': 0.25825460241917186, 'maui onion': 0.25559418564078545, 'wildflower honey': 0.2553300550540712, 'maple syrup': 0.25261499874355}
most different =  {'chip plain tortilla': -0.30218823738480094, 'steamer': -0.30022764190734846, 'frozen onion rings': -0.2995276056289855, 'fontina cheese': -0.2905440404819733, 'chutney': -0.2834118452000336, 'boneless skinless chicken': -0.27773726487031924, 'Azteca Flour Tortillas': -0.27545403007593366, 'no salt added chicken broth': -0.2744336296233577, 'rose essence': -0.26962961512622957, 'rib-eye roast': -0.26949861968681665}


In [111]:
compare(71,lstm128,10)

black tea
most similar =  {'beef broth': 0.3257762389554661, 'honey mustard': 0.3232873874878513, 'cornbread crumbs': 0.31892353911829197, 'peaches in heavy syrup': 0.30199684793397574, 'amaretti': 0.29702671152164756, 'butterscotch filling': 0.2901087746069962, 'vanilla low-fat ic cream': 0.28647609634874555, 'low-fat bottled italian dressing': 0.2833686948417287, 'reduced sodium chicken broth': 0.26513409487442, 'Mexican beer': 0.2634306176367107}
most different =  {'ibarra': -0.33301900265293694, 'granulated garlic': -0.311303846090432, 'george dickel': -0.28926822320945333, 'prepared horseradish': -0.28857268856137785, 'low-fat sweetened condensed milk': -0.2871094807966323, 'wheatberries': -0.283272610323107, 'tomato sauce': -0.27791103884235874, 'veal chops': -0.27787992936550165, 'corn niblets': -0.2776132191841156, 'fresh cheese': -0.2745132010567357}


Node2Vec

In [112]:
compare(956,node2vecp1q10,10)

oysters
most similar =  {'shrimp stock': 0.9727365340355812, 'smoked ham hocks': 0.972431925099041, 'lobster': 0.969535368905853, 'baby carrots': 0.9688908268180506, 'roux': 0.9682998047576298, 'crab': 0.966191784331447, 'onion tops': 0.9652386314346089, 'bay scallops': 0.9652128350572994, 'duck drumsticks': 0.9630989593240639, 'file powder': 0.9621494903824921}
most different =  {'pot pie': 0.06338735106406526, 'lemon lime beverage': 0.1927555505567268, 'food paste color': 0.234096237949611, 'dried chipotle pepper': 0.2571191859121467, 'spring mix': 0.2753309649350241, 'orange soda': 0.28440215761090754, 'custard': 0.2918262207509767, 'blackberry brandy': 0.33286064650878583, 'muesli': 0.36997439999986187, 'mini pepperoni slices': 0.39131996934092417}


In [113]:
compare(2945,node2vecp1q10,10)

water
most similar =  {'salt': 0.9507025988898223, 'vegetable oil': 0.9473874900025736, 'chicken legs': 0.9211048407255304, 'gray salt': 0.9125490763209293, 'fermented bean paste': 0.9105133262864954, 'liver': 0.910123109894362, 'chicken stock cubes': 0.9096208479959479, 'lychees': 0.9095846465017663, 'jerk marinade': 0.9094002929368595, 'fat free lemon curd': 0.9093667480930845}
most different =  {'lemon lime beverage': 0.1332463658181653, 'pot pie': 0.1435116522405703, 'orange soda': 0.2707331252592644, 'food paste color': 0.27237026930336594, 'custard': 0.29374238096313104, 'spring mix': 0.2944619695643378, 'blackberry brandy': 0.3011717672653678, 'dried chipotle pepper': 0.3199197460928213, 'mini pepperoni slices': 0.3288413300893504, 'muesli': 0.3657493397991774}


In [114]:
compare(3934,node2vecp1q10,10)

beef
most similar =  {'cooking oil': 0.9599604677512701, 'stock': 0.9574579464248757, 'chili pepper': 0.9496183282428555, 'vegetables': 0.9467199423990318, 'hot pepper': 0.9459413850576688, 'crushed garlic': 0.9447352007022617, 'egg noodles': 0.9440501914041809, 'corn flour': 0.9433176063935738, 'veggies': 0.9426326546749478, 'rock sugar': 0.9406197053314157}
most different =  {'pot pie': 0.008570119031002804, 'dried chipotle pepper': 0.20570669131750147, 'orange soda': 0.2374336074258251, 'food paste color': 0.24219193120202798, 'lemon lime beverage': 0.24655094904218144, 'Jell-O Gelatin Dessert': 0.2682875717536229, 'spring mix': 0.2727382052123918, 'muesli': 0.3124216871997025, 'blackberry brandy': 0.32322375045625484, 'custard': 0.3481990095322246}


In [115]:
compare(71,node2vecp1q10,10)

black tea
most similar =  {'coca-cola': 0.9960111431988234, 'minced meat': 0.9949027552287408, 'white flour': 0.9944707651338714, 'dried orange peel': 0.9943042726850589, 'straight bourbon whiskey': 0.9942489291508018, 'cold milk': 0.9940753016713676, '7 Up': 0.9940222640930099, 'crushed graham crackers': 0.9939634604658942, 'rich chicken stock': 0.9939432620666319, 'chocolate bars': 0.9938186585691158}
most different =  {'pot pie': 0.09676875765493005, 'food paste color': 0.19811228354302465, 'spring mix': 0.23679780537180073, 'lemon lime beverage': 0.2902902468466686, 'blackberry brandy': 0.30461374729637064, 'Jell-O Gelatin Dessert': 0.3246899930357494, 'orange soda': 0.33176463280229324, 'dried chipotle pepper': 0.3417767253441464, 'custard': 0.3508132834630843, 'mini pepperoni slices': 0.3553099883982496}


Word2Vec

In [116]:
compare(956,word2vec,10)

oysters
most similar =  {'thick-cut bacon': 0.943112839678314, 'savory': 0.9406266352233859, 'corn grits': 0.9398516701893307, 'ham steak': 0.9369362393063532, 'port': 0.9345254226325422, 'rib': 0.9342973223042632, 'shucked oysters': 0.9333980895781195, 'gumbo file': 0.930574538426432, 'canned beef broth': 0.9301310563429425, 'fryer chickens': 0.9288773178160609}
most different =  {'plain seltzer': -0.289447538252755, 'organic granulated sugar': -0.2120368409709209, 'carbonated beverages': -0.20216399322704, 'grassfed beef': -0.1989113412617365, 'sweet turkey sausage': -0.18646435474004946, 'english breakfast tea bags': -0.18512588231031069, 'mozzarella string cheese': -0.18369786917698464, 'raw peeled prawns': -0.1801734595730252, 'chicken in water': -0.17453909812208498, 'lipton onion soup mix': -0.17204357439412177}


In [117]:
compare(2945,word2vec,10)

water
most similar =  {'hot water': 0.7456585484572232, 'cold water': 0.6478383701496664, 'meat': 0.5792389117584638, 'vegetable stock': 0.5768585537254886, 'extra large shrimp': 0.57599311660342, 'boiling water': 0.5753425989511542, 'pork stock': 0.5743322166718592, 'boneless chicken thighs': 0.5721241291413036, 'yams': 0.5700997124870766, 'fat': 0.5690582733735676}
most different =  {'hakusai': -0.2605743781891103, 'treviso radicchio': -0.21661478487743382, 'ground poppy seed': -0.2136481831882251, 'butterscotch filling': -0.21364351205597096, 'bai cai': -0.2037984957351341, 'egg roll skins': -0.20096035842688972, 'fresh lavender': -0.1950996708815254, 'raw peeled prawns': -0.19490549902365925, 'avocado dressing': -0.19209085635690637, 'knorr homestyl stock   beef': -0.18201972739681155}


In [118]:
compare(3934,word2vec,10)

beef
most similar =  {'soup': 0.8818879862864586, 'boneless chicken breast': 0.880584347567597, 'boneless chicken thighs': 0.8775385990793682, 'veggies': 0.8750209153704601, 'crab': 0.8494961508890447, 'pork chops': 0.8492052195607446, 'shrimp shells': 0.8479453925401622, 'greens': 0.8446080306100502, 'calamansi': 0.8388486327510606, 'chili pepper flakes': 0.8301996459691434}
most different =  {'crumpet': -0.21762833677539656, 'bai cai': -0.21014177036721363, 'asakusa nori': -0.20977332353584618, 'non dairy yogurt': -0.20760134150345208, 'Flora Original': -0.20020575316202235, 'carbonated beverages': -0.1914197622922052, 'fresh lavender': -0.18325794871253473, 'osetra caviar': -0.1788947192231162, 'Manischewitz Matzo Meal': -0.1749339535537169, 'tomate verde': -0.1697905508814066}


In [119]:
compare(71,word2vec,10)

black tea
most similar =  {'cassava': 0.9624087345995597, 'seltzer water': 0.9620173572087994, 'flan': 0.9606649718635595, 'chilled prosecco': 0.9577636742822401, 'cacao nibs': 0.9562265433565511, 'southern comfort': 0.9549010016670608, 'candied ginger': 0.9542447560231286, 'tea leaves': 0.9536891938338613, 'cream of coconut': 0.9535620571021741, 'chocolate covered english toffee': 0.9534323882498903}
most different =  {'plain seltzer': -0.2218491054856095, 'smithfield ham': -0.21223085202242148, 'carbonated beverages': -0.19978185703601828, 'butter flavor shortening': -0.19555851098544377, 'manouri': -0.1931570017098595, 'chourico': -0.1863034041147966, 'lox': -0.18249967420545507, 'Camellia Red Kidney Beans': -0.17541615890663934, 'table wine': -0.16955649029217362, 'pangasius': -0.16310622761726531}


This comparaison is purely qualitative and may not reflect how each embedding perform in defferent method. It's seems that the SVD, SVD weighted, Node2Vec are giving result that make the most sense.

The embedding coming from the first layer of the LSTM classifier seems to be the worst.